# 导包

In [5]:
!wc -l ./predictions/final_submission_20190811.csv
print()
!head ./predictions/final_submission_20190811.csv
!head /home/kesci/input/bytedance/bytedance_contest.final_2.csv
print()
!tail ./predictions/final_submission_20190811.csv
!tail /home/kesci/input/bytedance/bytedance_contest.final_2.csv

100000000 ./predictions/final_submission_20190811.csv

1,1,0.53125741515603
1,2,0.4392272420196072
1,3,0.5879643281299144
1,4,0.576160912203239
1,5,0.5052100835619331
1,6,0.4324311875604571
1,7,0.4066308458541816
2,1,0.5768602948272425
2,2,0.6554118653967363
2,3,0.5325662217303531
1,128 770 122 1192,1,770 36 1192 8 33 10048 122 193 469 31 37	
1,128 770 122 1192,2,354 770 1192 40 9315 15 3545 14 522 3159 122 1645 4626 31 37	
1,128 770 122 1192,3,770 69380 1845 34 644 115 10320 11843 12348	
1,128 770 122 1192,4,770 770 4241 2131 7036 122 1192 851 2507	
1,128 770 122 1192,5,770 14692 3933 3194 27 10320 128 4346 1192	
1,128 770 122 1192,6,770 36 39 1192 266 108 5229 192 122 26831 11	
1,128 770 122 1192,7,770 36 10048 122 13 449 1192 33 708 30 37	
2,66 64 123 848,1,3589 458 848 66 26 123 848 388 17779 29794 27 1282 2435 389	
2,66 64 123 848,2,3589 458 848 66 26 123 848 388 17779 328 274 82424	
2,66 64 123 848,3,5218 1257 31 4712 6030 27 47 267 659 1392 3589 66 64 123 27 5692 848 819	

10318

In [13]:
# !pip install fuzzywuzzy -i https://pypi.tuna.tsinghua.edu.cn/simple
# !pip install jellyfish -i https://pypi.tuna.tsinghua.edu.cn/simple
# !pip install pyemd -i https://pypi.tuna.tsinghua.edu.cn/simple
# !pip install python-levenshtein -i https://pypi.tuna.tsinghua.edu.cn/simple

    100% |████████████████████████████████| 143kB 7.6MB/s eta 0:00:01
  Running setup.py bdist_wheel for jellyfish ... done
  Stored in directory: /home/kesci/.cache/pip/wheels/bc/b7/78/6736d761d7635d2af9579e040f342b5482850d856d26cbefa3
Successfully built jellyfish
    100% |████████████████████████████████| 92kB 7.5MB/s eta 0:00:01
  Running setup.py bdist_wheel for pyemd ... done
  Stored in directory: /home/kesci/.cache/pip/wheels/94/20/c1/ccdf0e9878f5c76def850603e62a572746036829f8353804bd
Successfully built pyemd
    100% |████████████████████████████████| 51kB 5.9MB/s eta 0:00:01
  Running setup.py bdist_wheel for python-levenshtein ... done
  Stored in directory: /home/kesci/.cache/pip/wheels/ef/af/8f/b3250804480b8d14ca55d436129a2fb53798a0ae9287b686c0
Successfully built python-levenshtein


In [1]:
%%time

import os
import sys
import math
import pickle
import collections
import gc
import joblib
import gzip
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import lightgbm as lgb
from tqdm import tqdm
# from fuzzywuzzy import fuzz
from datetime import datetime
from multiprocessing import Pool
# from jellyfish import jaro_distance, jaro_winkler
from scipy.spatial.distance import cosine, euclidean, cityblock

import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, FastText, KeyedVectors
from gensim.models.word2vec import Word2Vec, PathLineSentences, LineSentence

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import roc_auc_score, precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib

import networkx as nx
from networkx.readwrite.gpickle import write_gpickle, read_gpickle

import warnings
warnings.filterwarnings('ignore', category=Warning)

CPU times: user 1.3 s, sys: 596 ms, total: 1.9 s
Wall time: 1.45 s


# 常量初始化

In [2]:
%%time
test_data_path = '/home/kesci/input/bytedance/test_final_part1.csv'
train_data_path = './split/10kw.csv'
final_data_path = '/home/kesci/input/bytedance/bytedance_contest.final_2.csv'

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.72 µs


# 数据预处理

## 获取query和title的set集合——train

In [16]:
%%time
def get_unique(filepath):
    print('-----------------reading {}-----------------'.format(filepath))
    data = pd.read_csv(filepath, chunksize=1000000, header=None)
    data.columns = {'query_id', 'query', 'title_id', 'title', 'label'}
    sentences = set()
    query = set()
    title = set()
    print('------------------parse data------------------')
    i = 0
    for chunk in data:
        if i%10 == 0: print(i)
        i+=1
        query |= set(chunk[1].unique().tolist())
        title |= set(chunk[3].unique().tolist())
    sentences = query | title
    
    print('unique query size', query.__len__())
    print('unique title size', title.__len__())
    print('unique sentences size', sentences.__len__())
    # print('---------------Writing to csv file---------------')

    # with open(handled_path + 'sentences.csv', 'w', encoding='utf-8') as f:
    #     for item in sentences:
    #         f.write(item)
    #         f.write('\n')
    return query, title, sentences
test_query, test_title, test_sentence = get_unique(test_data_path)
train_query, train_title, train_sentence = get_unique(train_data_path)
final_query, final_title, final_sentence = get_unique(final_data_path)

sentence = test_sentence | train_sentence | final_sentence
print(sentence.__len__())
with open('./handled_data/all_sentences.csv', 'w', encoding='utf-8') as f:
    for item in sentence:
        f.write(item)
        f.write('\n')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 10.3 µs


## wordcount

In [ ]:
word_counter = collections.Counter()

def word_count(filepath):
    data = pd.read_csv(filepath, chunksize=1000000, header=None)
    for chunk in tqdm(data):
        for item in chunk[1].unique():
            for word in item.split():
                word_counter[word] += 1
        for item in chunk[3].tolist():
            for word in item.split():
                word_counter[word] += 1
                
def get_weight(count, eps=10000, min_count=2):
    return 0 if count < min_count else 1/(count + eps)
    
word_count(test_data_path)
word_count(train_data_path)
word_count(final_data_path)

weights = {word : get_weight(count) for word, count in word_counter.items()}

In [ ]:
def word_shares(row):
    q1_list = str(row['question1']).lower().split()
    q1 = set(q1_list)
    q1words = q1.difference(stops)
    if len(q1words) == 0:
        return '0:0:0:0:0:0:0:0'

    q2_list = str(row['question2']).lower().split()
    q2 = set(q2_list)
    q2words = q2.difference(stops)
    if len(q2words) == 0:
        return '0:0:0:0:0:0:0:0'

    words_hamming = sum(1 for i in zip(q1_list, q2_list) if i[0]==i[1])/max(len(q1_list), len(q2_list))

    q1stops = q1.intersection(stops)
    q2stops = q2.intersection(stops)

    q1_2gram = set([i for i in zip(q1_list, q1_list[1:])])
    q2_2gram = set([i for i in zip(q2_list, q2_list[1:])])

    shared_2gram = q1_2gram.intersection(q2_2gram)

    shared_words = q1words.intersection(q2words)
    shared_weights = [weights.get(w, 0) for w in shared_words]
    q1_weights = [weights.get(w, 0) for w in q1words]
    q2_weights = [weights.get(w, 0) for w in q2words]
    total_weights = q1_weights + q1_weights

    R1 = np.sum(shared_weights) / np.sum(total_weights) #tfidf share
    R2 = len(shared_words) / (len(q1words) + len(q2words) - len(shared_words)) #count share
    R31 = len(q1stops) / len(q1words) #stops in q1
    R32 = len(q2stops) / len(q2words) #stops in q2
    Rcosine_denominator = (np.sqrt(np.dot(q1_weights,q1_weights))*np.sqrt(np.dot(q2_weights,q2_weights)))
    Rcosine = np.dot(shared_weights, shared_weights)/Rcosine_denominator
    if len(q1_2gram) + len(q2_2gram) == 0:
        R2gram = 0
    else:
        R2gram = len(shared_2gram) / (len(q1_2gram) + len(q2_2gram))
    return '{}:{}:{}:{}:{}:{}:{}:{}'.format(R1, R2, len(shared_words), R31, R32, R2gram, Rcosine, words_hamming)

# 生成无向图

## 准备数据——无向图生成

In [1]:
%%time
print('start reading seq2id')
seq2id = {}
with open('./handled_data/all_sentences.csv', 'r', encoding='utf-8') as f:
    for i, sentence in enumerate(f.readlines()):
        seq2id[sentence.strip()] = i
print('end reading seq2id')
print('seq2id with length --->',len(seq2id))

start reading seq2id
end reading seq2id
seq2id with length ---> 87907776
CPU times: user 1min 31s, sys: 14.5 s, total: 1min 45s
Wall time: 2min 4s


## 文本转ID

In [26]:
%%time
print(datetime.now())
def transform(filepath):
    data = pd.read_csv(filepath, chunksize=1000000, header=None)
    data.columns = {'query_id', 'query', 'title_id', 'title', 'label'}
    result = []
    print('start transform data')
    idx = 0
    for chunk in data:
        if idx%10==0: print(idx)
        idx += 1
        result.append(pd.DataFrame({0:chunk[1].apply(lambda x : seq2id[x.strip()]), 
                                    1:chunk[3].apply(lambda x : seq2id[x.strip()])}))
    return result
# id = pd.concat(transform(test_data_path))
print(id.describe())
id.to_csv('./handled_data/id_test.csv', header=None, index=False)
!wc -l ./handled_data/id_test.csv

2019-08-04 18:25:18.316136
start transform data
0
10
                  0             1
count  2.000000e+07  2.000000e+07
mean   5.122025e+07  6.027587e+07
std    2.944711e+07  2.793264e+07
min    1.060000e+02  0.000000e+00
25%    2.575488e+07  3.862078e+07
50%    5.130830e+07  6.404053e+07
75%    7.676274e+07  8.443098e+07
max    1.019757e+08  1.019757e+08
CPU times: user 1min 18s, sys: 1.86 s, total: 1min 20s
Wall time: 1min 28s


## 生成无向图

In [3]:
%%time
print(datetime.now())
G = nx.Graph()
    
def make_graph_parallel():
    CHUNK_SIZE = 1000000
    # test_data = pd.read_csv('./handled_data/id_test.csv', chunksize=CHUNK_SIZE, header=None)
    # test_data.columns = {'query_id', 'query', 'title_id', 'title', 'label'}
    
    # print('generate test graph')
    # idx = 0
    # print(datetime.now())
    # for chunk in test_data:
    #     print(datetime.now(),'--->',idx)
    #     idx += 1 
    #     ziped = list(zip(chunk[0], chunk[1]))
    #     G.add_edges_from(ziped)
        
    #     del chunk
    #     del ziped
    # print('finish generate test graph, temporaray graph size ->', G.number_of_nodes(), G.number_of_edges())
    # del test_data
    # gc.collect()
    
    data = pd.read_csv('./handled_data/id_train.csv', chunksize=CHUNK_SIZE, header=None)
    data.columns = {'query_id', 'query', 'title_id', 'title', 'label'}

    print('generate train graph')
    idx = 0
    # for chunk in data:
    for i in range(100):
        chunk = data.get_chunk()
        if i < 70: continue
        print(datetime.now(), '--->', idx)
        idx += 1
        ziped = list(zip(chunk[0], chunk[1]))
        G.add_edges_from(ziped)
        
        del chunk
        del ziped
    del data
    gc.collect()
    print('finish generate train graph, temporaray graph size ->', G.number_of_nodes(), G.number_of_edges())
    
    data = pd.read_csv('./handled_data/id_final.csv', chunksize=CHUNK_SIZE, header=None)
    data.columns = {'query_id', 'query', 'title_id', 'title', 'label'}
    print('generate final data graph')
    idx = 0
    for chunk in data:
        print(datetime.now(), '--->', idx)
        idx += 1
        ziped = list(zip(chunk[0], chunk[1]))
        G.add_edges_from(ziped)
        
        del chunk
        del ziped
    print('finish generate final graph, temporaray graph size ->', G.number_of_nodes(), G.number_of_edges())
   
    del data
    gc.collect()
    print('save graph')
    write_gpickle(G, './handled_data/final_graph.pkl')
    
make_graph_parallel()

2019-08-04 20:09:48.426697
generate train graph
2019-08-04 20:10:11.280335 ---> 0
2019-08-04 20:10:16.641768 ---> 1
2019-08-04 20:10:21.355543 ---> 2
2019-08-04 20:10:27.104319 ---> 3
2019-08-04 20:10:31.788189 ---> 4
2019-08-04 20:10:36.931844 ---> 5
2019-08-04 20:10:40.341101 ---> 6
2019-08-04 20:10:46.926877 ---> 7
2019-08-04 20:10:53.250733 ---> 8
2019-08-04 20:10:56.573122 ---> 9
2019-08-04 20:10:59.920231 ---> 10
2019-08-04 20:11:07.224468 ---> 11
2019-08-04 20:11:10.635839 ---> 12
2019-08-04 20:11:14.068377 ---> 13
2019-08-04 20:11:24.505138 ---> 14
2019-08-04 20:11:27.838067 ---> 15
2019-08-04 20:11:31.123212 ---> 16
2019-08-04 20:11:34.418728 ---> 17
2019-08-04 20:11:44.396497 ---> 18
2019-08-04 20:11:47.722042 ---> 19
2019-08-04 20:11:51.068181 ---> 20
2019-08-04 20:11:54.431964 ---> 21
2019-08-04 20:11:57.809854 ---> 22
2019-08-04 20:12:10.010839 ---> 23
2019-08-04 20:12:13.399495 ---> 24
2019-08-04 20:12:16.806213 ---> 25
2019-08-04 20:12:20.229955 ---> 26
2019-08-04 20:12:

## 生成pagerank模型

In [5]:
%%time
print(datetime.now())
graph = read_gpickle('./handled_data/final_graph.pkl')
print(datetime.now(),'---> end load graph')
page_rank = nx.pagerank_scipy(graph)
print('end generate pagerank value')
pickle.dump(page_rank, open('./handled_data/pagerank_final.model', 'wb'))

2019-08-04 20:26:20.708101
end generate pagerank value
CPU times: user 21min, sys: 51.1 s, total: 21min 52s
Wall time: 21min 51s


## 生成HITS模型

In [1]:
%%time
print(datetime.datetime.now())
graph = joblib.load('./models/un_direct_graph.model')
print('end load graph')
hits_h, hits_a = nx.hits(graph)
print('end generate HITS value')
joblib.dump(hits_h, './models/hub_value.model')
joblib.dump(hits_a, './models/authority_value.model')

2019-07-27 03:39:19.758518
end load graph


## 数据准备——特征生成

In [14]:
%%time
# graph = read_gpickle('./handled_data/final/final_graph.pkl')
# print('end load graph')
# seq2id = {}
# with open('./handled_data/all_sentences_verseion_2.csv', 'r', encoding='utf-8') as f:
#     for i, sentence in enumerate(f.readlines()):
#         seq2id[sentence.strip()] = i
# print('end seq2id')
def gen_degrees():
    max_degrees = {}
    edges = graph.edges()
    for edge in edges:
        for n in edge:
            max_degrees[n] = max_degrees.get(n, 0) + 1
    return max_degrees

def gen_components():
    max_components = {}
    components = nx.connected_components(graph)
    for component in components:
        for n in component:
            max_components[n] = max(max_components.get(n, 0), len(component))
    return max_components

def gen_hits():
    hits_h, hits_a = nx.hits(graph, max_iter=500)
    return hits_h, hits_a

# max_degrees = gen_degrees()
# pickle.dump(max_degrees, open('./handled_data/max_degree_final', 'wb'))
# print('end degree')
# max_components = gen_components()
# pickle.dump(max_components, open('./handled_data/max_components_final', 'wb'))
# print('end components')
# del graph
# gc.collect()

max_degrees = pickle.load(open('./handled_data/final/max_degree_final', 'rb'))
max_components = pickle.load(open('./handled_data/final/max_components_final', 'rb'))
page_rank = pickle.load(open('./handled_data/final/pagerank_final.model', 'rb'))
# print('end pagerank')
# hits_h, hits_a = gen_hits()

# max_degree max_components
def calculate_statistics(row):
    return [max_degrees[row[0]], max_degrees[row[1]], max_components[row[0]]]      
     
# PageRank     
def calculate_pagerank(row):
    return [page_rank[row[0]] * 1e6, page_rank[row[1]] * 1e6]       

# Hits
def calculate_hits(row):
    hits_h_1 = hits_h[row[0]] * 1e6
    hits_a_1 = hits_a[row[0]] * 1e6
    hits_h_2 = hits_h[row[1]] * 1e6
    hits_a_2 = hits_a[row[1]] * 1e6
    return [hits_h_1, hits_a_1, hits_h_2, hits_a_2]

# ShortestPath
def calculate_shortestpath(row):
    graph.remove_edge(row[0], row[1])
    if nx.has_path(graph, row[0], row[1]):
        shortest_path = len(nx.shortest_path(graph, row[0], row[1]))
    else:
        shortest_path = -1
    graph.add_edge(row[0], row[1])
    return [shortest_path]

# Neighbour
def calculate_neighbour(row):
    neighbor_1 = set(graph.neighbors(row[0]))
    neighbor_2 = set(graph.neighbors(row[1]))
    return [len(neighbor_1), len(neighbor_2), len(neighbor_1 | neighbor_2)]

CPU times: user 52.5 s, sys: 12.3 s, total: 1min 4s
Wall time: 1min 5s


## 生成图特征

In [17]:
%%time
gc.collect()
CHUNKSIZE = 1000000
data = pd.read_csv('./handled_data/id/id_train.csv', chunksize=CHUNKSIZE, header=None)
data.columns = {'query_id', 'query', 'title_id', 'title', 'label'}
pool = Pool()
res = []
idx = 0
for i in range(70):
    data.get_chunk()
print('start generate feature')
# for chunk in data:
for i in range(30):
    chunk = data.get_chunk()
    print(idx)
    idx += 1
    df = []
    ziped = list(zip(chunk[0], chunk[1]))
    # ziped = list(zip(chunk[1].apply(lambda x:seq2id[x.strip()]).tolist(), 
    #                                 chunk[3].apply(lambda x:seq2id[x.strip()]).tolist()))
                                    
    df.append(pd.DataFrame(pool.map(calculate_statistics, ziped)))
    df.append(pd.DataFrame(pool.map(calculate_pagerank, ziped)))
    
    # df.append(pd.DataFrame(pool.map(calculate_hits, ziped)))
    # df.append(pd.DataFrame(pool.map(calculate_neighbour, ziped)))
    
    res.append(pd.concat(df, axis=1))
    del ziped
    del chunk
    del df
    gc.collect()
    
graph_features = pd.concat(res)
graph_features.to_csv('./handled_data/train_feature/train_feature_graph_final.csv', header=None, index=False)
print(graph_features.shape)
print(graph_features.describe())
del graph_features
gc.collect()

start generate feature
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
(30000000, 5)
                  0             1             2             0             1
count  3.000000e+07  3.000000e+07  3.000000e+07  3.000000e+07  3.000000e+07
mean   1.274293e+02  1.106532e+01  5.754290e+07  1.234310e+00  2.669189e-02
std    3.819820e+03  3.443744e+02  5.868692e+06  3.824809e+01  3.425827e+00
min    1.000000e+00  1.000000e+00  2.000000e+00  2.574202e-03  2.541272e-03
25%    7.000000e+00  1.000000e+00  5.814144e+07  4.419803e-02  3.981151e-03
50%    1.600000e+01  3.000000e+00  5.814144e+07  8.919203e-02  6.508193e-03
75%    2.000000e+01  7.000000e+00  5.814144e+07  1.496774e-01  1.367823e-02
max    1.289980e+05  1.289980e+05  5.814144e+07  1.291623e+03  1.291623e+03
CPU times: user 6min 29s, sys: 21.1 s, total: 6min 50s
Wall time: 7min 1s


14

# 一般特征

## 准备数据

In [5]:
%%time
dictionary = Dictionary.load('./models/word_dict.dict') # gensim Dictionary
print('loaded dict')
w2v_model = KeyedVectors.load('/home/kesci/work/word_vectors/w2v-300d-new.txt')
print('loaded word2vec')
tfidf_model = TfidfModel.load('./models/tfidf.model')
print('loaded TF-IDF')
fasttext_model = FastText.load('./word_vectors/fast-300d-new.txt')
# tfidf = joblib.load(model_path + 'scikit_tfidf_model.m')
print('----------------initialized models----------------')

def init_prob():
    CHUNK_SIZE = 1000000
    reader = pd.read_csv('./split/10kw.csv',
                    chunksize=CHUNK_SIZE,
                    header=None,
                    names=['query_id','query','query_title_id','title','label'])
        
    totalCounter = collections.Counter()
    posCounter = collections.Counter()
    
    idx = 0
    for chunk in reader:
        # if idx == 10: break
        print(idx)
        idx += 1
        query = chunk['query'].apply(lambda x : x.split()).values.tolist()
        title = chunk['title'].apply(lambda x : x.split()).values.tolist()
        label = chunk['label'].values.tolist()
        for i in range(CHUNK_SIZE):
            for word in query[i]:
                totalCounter[word] += 1
                if label[i] == 1:
                    posCounter[word] += 1
            for word in title[i]:
                totalCounter[word] += 1
                if label[i] == 1:
                    posCounter[word] += 1
    prob = {}
    for key, value in posCounter.items():
        prob[key] = value / totalCounter[key]
    return prob   
# pos_prob = init_prob() # 需要计算出这个单词出现之后label为1的概率
# file = open(handled_path + 'pos_10kw.pkl', 'wb+')
# pickle.dump(pos_prob, file)

# file = open('./handled_data/pos_10kw.pkl', 'rb+')
# pos_prob = pickle.load(file)
# print('----------------initialized prob----------------')

loaded dict
loaded word2vec
loaded TF-IDF
----------------initialized models----------------
CPU times: user 9.07 s, sys: 4.41 s, total: 13.5 s
Wall time: 27.4 s


## powerful word

In [3]:
%%time
# words_power = {}
# def parse(row):
#     label = int(row[2])
#     q1_words = row[0]
#     q2_words = row[1]
#     q1_words = set(q1_words)
#     q2_words = set(q2_words)
#     all_words = q1_words |q2_words
#     for word in all_words:
#         if word not in words_power:
#             words_power[word] = [0. for i in range(5)]
#         words_power[word][0] += 1.

#         if ((word in q1_words) and (word not in q2_words)) or ((word not in q1_words) and (word in q2_words)):
#             words_power[word][1] += 1.
#             if 1 == label:
#                 words_power[word][2] += 1.
#         if (word in q1_words) and (word in q2_words):
#             words_power[word][3] += 1.
#             if 1 == label:
#                 words_power[word][4] += 1.
# def generate_powerful_word(filepath):
#     data = pd.read_csv(filepath, chunksize=1000000, header=None)

#     for chunk in tqdm(data):
#         ziped = list(zip(chunk[1].apply(lambda x:x.split()).tolist(),
#                         chunk[3].apply(lambda x:x.split()).tolist(),
#                             chunk[4]))
#         for row in ziped:
#             parse(row)
# generate_powerful_word('/home/kesci/input/bytedance/train_final.csv')
# for word in tqdm(words_power):
#     if words_power[word][1] > 1e-6:
#         words_power[word][2] /= words_power[word][1]
#     words_power[word][1] /= words_power[word][0]
#     if words_power[word][3] > 1e-6:
#         words_power[word][4] /= words_power[word][3]
#     words_power[word][3] /= words_power[word][0]
# pickle.dump(words_power, open('./handled_data/powerful_words', 'wb'))


words_power = pickle.load(open('./handled_data/powerful_words', 'rb'))

CPU times: user 2.1 s, sys: 204 ms, total: 2.3 s
Wall time: 2.3 s


## 并行统计特征

In [17]:
# 获取 sentence 的tfidf值
def getTFIDF(words):
    tfidf = tfidf_model[dictionary.doc2bow(words)]
    dic = {word:0 for word in words}
    sum_weight = 1e-9
    for idx, val in tfidf:
        dic[dictionary[idx]] = val
        sum_weight += val
        
    return dic, sum_weight

# query与title都出现过的word的词数除以总词数
def concurrence(data):
    query = data[0]
    title = data[1]
    query_words = {}
    title_words = {}
    for word in query:
        query_words[word] = query_words.get(word, 0) + 1
    for word in title:
        title_words[word] = title_words.get(word, 0) + 1

    shared_query_word = sum([query_words[w] for w in query if w in title])
    shared_titel_word = sum([title_words[w] for w in title if w in query])
    total = sum(query_words.values()) + sum(title_words.values())

    if 1e-6 > total:
        return [0]
    else:
        return [1.0 * (shared_titel_word + shared_query_word) / total]

# 编辑距离，词粒度
def levenshteinDistance(data):
    query = data[0]
    title = data[1]
    
    len_query = len(query) + 1
    len_title = len(title) + 1
    dp = [[0] * len_title] * len_query

    for i in range(1, len_query):
        for j in range(1, len_title):
            deletion = dp[i-1][j] + 1
            insertion = dp[i][j-1] + 1
            substitution = dp[i-1][j-1]
            if query[i-1] != title[j-1]:
                substitution += 1
            dp[i][j] = min(deletion, insertion, substitution)
    return [1 - dp[-1][-1] / max(len_title, len_query)]

# sorensen距离
def sorensenDistance(data):
    query = data[0]
    title = data[1]
    words = set(query) | set(title)

    query_dict = {}
    for word in query:
        query_dict[word] = query_dict.get(word, 0) + 1

    title_dict = {}
    for word in title:
        title_dict[word] = title_dict.get(word, 0) + 1

    total = sum(query_dict.values()) + sum(title_dict.values())

    diff = 0
    for word in words:
        diff += abs(query_dict.get(word, 0) - title_dict.get(word, 0))

    return [diff / total]
    
# query和title中的共现词 
# 计算某个词出现的时候label为1的概率，然后求所有词的概率乘积
def sameWord(data):
    unique_words = set(data[0]) | set(data[1])
    prob = 1
    for word in unique_words:
        prob *= pos_prob.get(word, 1)
    return [len(unique_words), prob]

# Dice Ochi
# 1 - 交集除以并集jaccard
def distance(data):
    query = data[0]
    title = data[1]
    intersection = len([x for x in query if x in title])
    l = len(query) + len(title)
    return [2*intersection / l, intersection / np.sqrt(l),
            1- (intersection / (l - intersection))]
            
def fuzzyDistance(data):
    return [
        fuzz.ratio(data[0], data[1]),
        fuzz.partial_ratio(data[0], data[1]),
        fuzz.token_sort_ratio(data[0], data[1]),
        fuzz.token_set_ratio(data[0], data[1]),
        fuzz.partial_token_sort_ratio(data[0], data[1]),
        w2v_model.wmdistance(data[0], data[1]),
        fasttext_model.wmdistance(data[0], data[1])
    ]

def powerfuleWord(data):
    rate_single = 1.0
    rate_double = 1.0
    query = set(data[0])
    title = set(data[1])
    
    share_words = query.intersection(title)
    all_diff = set(query.difference(title) | title.difference(query))
    for word in share_words:
        if word in words_power:
            rate_double *= (1.0 - words_power[word][4])
    for word in all_diff:
        if word in words_power:
            rate_single *= (1.0 - words_power[word][2])
    
    return [1-rate_single, 1-rate_double]

## 并行词嵌入特征

In [18]:
# 词向量距离
def word2vecDistance(data):
    query = data[0]
    title = data[1]
    vec_query = np.array([w2v_model[x] if x in w2v_model else [1e-9]*300 for x in query]).mean(axis=0)
    vec_title = np.array([w2v_model[x] if x in w2v_model else [1e-9]*300 for x in title]).mean(axis=0)

    cos_dis = 1 - cosine(vec_title, vec_query)
    euclidean_dis = 1 - euclidean(vec_title, vec_query)
    manhattan_dis = 1 - cityblock(vec_title, vec_query)
    
    vec_query = np.array([fasttext_model[x] if x in fasttext_model else [1e-9]*300 for x in query]).mean(axis=0)
    vec_title = np.array([fasttext_model[x] if x in fasttext_model else [1e-9]*300 for x in title]).mean(axis=0)

    cos_dis_fasttext = 1 - cosine(vec_title, vec_query)
    euclidean_dis_fasttext = 1 - euclidean(vec_title, vec_query)
    manhattan_dis_fasttext = 1 - cityblock(vec_title, vec_query)

    return [cos_dis, euclidean_dis, manhattan_dis, 
            cos_dis_fasttext, euclidean_dis_fasttext, manhattan_dis_fasttext] 

# 词向量距离，考虑TFIDF距离
# 获取重要程度在前topN的词的向量，求距离
# 只考虑tfidf权重的值的距离，cos、总和、平均
def w2vWeightDistance(data):
    topN = 3
    query = data[0]
    title = data[1]
    query_tfidf, query_weight = getTFIDF(query)
    title_tfidf, title_weight = getTFIDF(title)
    
    # word2vec距离，考虑TFIDF
    vec_query = np.sum(np.array([w2v_model[key] * value if key in w2v_model else [1e-9]*300 for key, value in query_tfidf.items()]), axis=0) / query_weight
    vec_title = np.sum(np.array([w2v_model[key] * value if key in w2v_model else [1e-9]*300 for key, value in title_tfidf.items()]), axis=0) / title_weight

    cos_dis = 1 - cosine(vec_query, vec_title)
    if np.isnan(cos_dis):
        cos_dis = 1
    euclidean_dis = 1 - euclidean(vec_query, vec_title)
    manhattan_dis = 1- cityblock(vec_query, vec_title)
    
    # fasttext距离，考虑TFIDF
    vec_query = np.sum(np.array([fasttext_model[key] * value if key in fasttext_model else [1e-9]*300 for key, value in query_tfidf.items()]), axis=0) / query_weight
    vec_title = np.sum(np.array([fasttext_model[key] * value if key in fasttext_model else [1e-9]*300 for key, value in title_tfidf.items()]), axis=0) / title_weight

    cos_dis_fasttext = 1 - cosine(vec_query, vec_title)
    if np.isnan(cos_dis_fasttext):
        cos_dis_fasttext = 1
    euclidean_dis_fasttext = 1 - euclidean(vec_query, vec_title)
    manhattan_dis_fasttext = 1- cityblock(vec_query, vec_title)
    
    # 获取重要程度在前topN的词
    query_keywords = sorted(zip(query_tfidf.keys(), query_tfidf.values()), key=lambda x : -x[1])[0 : min(topN, len(query_tfidf))]
    title_keywords = sorted(zip(title_tfidf.keys(), title_tfidf.values()), key=lambda x : -x[1])[0: min(topN, len(title_tfidf))]
    
    # 重要词的word2vec距离
    query_vector = np.mean(np.array([w2v_model[word] if word in w2v_model else [1e-9]*300 for word, value in query_keywords]), axis=0)
    title_vector = np.mean(np.array([w2v_model[word] if word in w2v_model else [1e-9]*300 for word, value in title_keywords]), axis=0)

    cos_dis_keyword = 1 - cosine(query_vector, title_vector)
    
    euclidean_dis_keyword = 1 - euclidean(query_vector, title_vector)
    manhattan_dis_keyword = 1 - cityblock(query_vector, title_vector)
    
    # 重要词的fasttext距离
    query_vector = np.mean(np.array([fasttext_model[word] if word in fasttext_model else [1e-9]*300 for word, value in query_keywords]), axis=0)
    title_vector = np.mean(np.array([fasttext_model[word] if word in fasttext_model else [1e-9]*300 for word, value in title_keywords]), axis=0)

    cos_dis_keyword_fasttext = 1 - cosine(query_vector, title_vector)
    euclidean_dis_keyword_fasttext = 1 - euclidean(query_vector, title_vector)
    manhattan_dis_keyword_fasttext = 1 - cityblock(query_vector, title_vector)
    
    # 只考虑tfidf权重的值的距离，cosine、sum、avg
    len1 = query_tfidf.__len__()
    len2 = title_tfidf.__len__()
    
    query_tfidf_value = np.array(list(query_tfidf.values()) + [1e-9]*max(len2-len1, 0))
    title_tfidf_value = np.array(list(title_tfidf.values()) + [1e-9]*max(len1-len2, 0))
    
    tfidf_distance = 1 - cosine(query_tfidf_value, title_tfidf_value)
    if np.isnan(tfidf_distance):
        tfidf_distance = 1
    query_tfidf_sum = np.sum(query_tfidf_value)
    title_tfidf_sum = np.sum(title_tfidf_value)
    query_tfidf_mean = np.mean(query_tfidf_value)
    title_tfidf_mean = np.mean(title_tfidf_value)

    return [cos_dis, euclidean_dis, manhattan_dis, 
            cos_dis_keyword, euclidean_dis_keyword, manhattan_dis_keyword,
            tfidf_distance, query_tfidf_sum, title_tfidf_sum, 
            query_tfidf_mean, title_tfidf_mean, 
            cos_dis_fasttext, euclidean_dis_fasttext, manhattan_dis_fasttext,
            cos_dis_keyword_fasttext, euclidean_dis_keyword_fasttext, manhattan_dis_keyword_fasttext]



## 并行生成特征

In [16]:
%%time
pool = Pool(8)
CHUNK_SIZE = 1000000
data = pd.read_csv('./split/3kw.csv', chunksize=CHUNK_SIZE, header=None)
data.columns = {'query_id', 'query', 'title_id', 'title', 'label'}

result = []
i = 0
for chunk in tqdm(data):
    i += 1
    # if i>10:break
    
    df = []
    # query = chunk[1].apply(lambda x : x.split()).tolist()
    # title = chunk[3].apply(lambda x : x.split()).tolist()
    
    query = chunk[1].tolist()
    title = chunk[3].tolist()
    d = list(zip(query, title))
    
    # df.append(pd.DataFrame(pool.map(concurrence, d)))
    # print(1)
    # df.append(pd.DataFrame(pool.map(levenshteinDistance, d)))
    # print(2)
    # df.append(pd.DataFrame(pool.map(sorensenDistance, d)))
    # print(3)
    # df.append(pd.DataFrame(pool.map(sameWord, d)))
    # print(4)
    # df.append(pd.DataFrame(pool.map(distance, d)))
    # print(5)
    # df.append(pd.DataFrame(pool.map(word2vecDistance, d)))
    # print(6)
    # df.append(pd.DataFrame(pool.map(w2vWeightDistance, d)))
    # print(7)
    # df.append(pd.DataFrame(pool.map(fuzzyDistance, d)))
    # print(8)
    # df.append(pd.DataFrame(pool.map(powerfuleWord, d)))
    # print(9)
    
    result.append(pd.concat(df, axis=1))

    del df
    del query
    del title
    del d
    del chunk
    gc.collect()

del data
result = pd.concat(result)
print(result.describe())
result.to_csv('./handled_data/train_feature/train_feature_jellyfish.csv', header=None, index=False)
# 释放资源
pool.close()
pool.terminate()
pool.join()
gc.collect()

30it [02:33,  5.08s/it]


                  0             1
count  3.000000e+07  3.000000e+07
mean   6.411826e-01  6.411826e-01
std    7.225124e-02  7.225124e-02
min    0.000000e+00  0.000000e+00
25%    5.931235e-01  5.931235e-01
50%    6.340817e-01  6.340817e-01
75%    6.841270e-01  6.841270e-01
max    1.000000e+00  1.000000e+00


# 特征相关性分析

## 相关系数

In [21]:
feature_name = ['concurrence', 'levenshteinDistance', 'sorensenDistance', 
        'sameWord', 'specialConcurrence',
        'diceDistance', 'ochiaiDistance', 'jaccardDistance', 
        'word2vecDistance_1', 'word2vecDistance_2', 'word2vecDistance_3', 
        'fasttextDistance_1', 'fasttextDistance_2', 'fasttextDistance_3', 
        'w2vWeightDistance_1', 'w2vWeightDistance_2', 'w2vWeightDistance_3',
        'w2vKeywordDistance_1', 'w2vKeywordDistance_2', 'w2vKeywordDistance_3',
        'TFIDFDistance_1', 'TFIDFDistance_2', 'TFIDFDistance_3', 
        'TFIDFDistance_4', 'TFIDFDistance_5', 
        'fasttextWeightDistance_1', 'fasttextWeightDistance_2', 'fasttextWeightDistance_3',
        'fasttextKeywordDistance_1', 'fasttextKeywordDistance_2', 'fasttextKeywordDistance_3',
        'pagerank_feature1', 'pagerank_feature2',
        'seq1_max_degree', 'seq2_max_degree', 'seq1_max_component',
        'seq1_neighbor', 'seq2_neighbor', 'total_neighbor',
        'DSSM_Feature', 'Deep_Model1', 'ARC', 'MVLSTM']
        
with open('./handled_data/correlation', 'r', encoding='utf-8') as f:
    for idx, line in enumerate(f.readlines()):
        tmp_data = list(map(lambda x:float(x), line.split(',')))
        for item in range(len(tmp_data)):
            if tmp_data[item] > 0.9 and item != idx:
                print(tmp_data[item], feature_name[idx], feature_name[item])

0.9776392403538859 sorensenDistance jaccardDistance
0.9191483643445837 diceDistance ochiaiDistance
0.9191483643445837 ochiaiDistance diceDistance
0.9776392403538859 jaccardDistance sorensenDistance
0.9790698526721849 word2vecDistance_1 fasttextDistance_1
0.9013797584783118 word2vecDistance_1 w2vWeightDistance_1
0.9982673082219394 word2vecDistance_2 word2vecDistance_3
0.971311990796078 word2vecDistance_2 fasttextDistance_2
0.9694247650601884 word2vecDistance_2 fasttextDistance_3
0.9982673082219394 word2vecDistance_3 word2vecDistance_2
0.9696246774668986 word2vecDistance_3 fasttextDistance_2
0.9677387984890892 word2vecDistance_3 fasttextDistance_3
0.9790698526721849 fasttextDistance_1 word2vecDistance_1
0.971311990796078 fasttextDistance_2 word2vecDistance_2
0.9696246774668986 fasttextDistance_2 word2vecDistance_3
0.9980665751664842 fasttextDistance_2 fasttextDistance_3
0.9694247650601884 fasttextDistance_3 word2vecDistance_2
0.9677387984890892 fasttextDistance_3 word2vecDistance_3
0.998

## heatmap

In [47]:
def plot_corr():
    corr = pd.read_csv('./handled_data/correlation', header=None)
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True
    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    g = sns.heatmap(corr, mask=mask, cmap=cmap, square=True, annot=True, fmt='0.2f')
    plt.figure(dpi=500)
    plt.show()
    plt.savefig('./try_test')
plot_corr()

<Figure size 432x288 with 2 Axes>

<Figure size 3000x2000 with 0 Axes>

<Figure size 432x288 with 0 Axes>

# RF

In [6]:
del x_train
del y_train
gc.collect()

7

In [5]:
rf = RandomForestClassifier(n_jobs=15,
                                n_estimators=150,
                                class_weight={0:1, 1:3},
                                verbose=1)
train_x = np.array(x_train.values).reshape(-1, 1).astype('int')
train_y = np.array(y_train.values).reshape(-1, 1).astype('int')

test_x = np.array(x_test.values).reshape(-1, 1).astype('int')
test_y = np.array(y_test.values).reshape(-1, 1).astype('int')
print('-------------------start train-------------------')
rf.fit(train_x, train_y)
pred_rf = rf.predict_proba(test_x)
auc_score = roc_auc_score(test_y, pred_rf)
acc_score = precision_score(test_y, pred_rf)
print('auc_score:', auc_score)
print('acc_score:', acc_score)

-------------------start train-------------------


[Parallel(n_jobs=15)]: Using backend ThreadingBackend with 15 concurrent workers.


# XGBoost

In [19]:
print(train.get_label())

[]


In [18]:
%%time
# train, test = train_test_split(total, test_size=0.1, random_state=20, stratify=total[label_idx])
# print('end split')
# param = {'objective': 'binary:logistic'}
# train = xgb.DMatrix(train)
# test = xgb.DMatrix(test)
# print(train.num_col())
# print(train.num_row())

bst = xgb.train(param, train, 2)

# pred_train = bst.predict(train)
# acc_score = accuracy_score(train.get_label(), pred_train)
# print(acc_score)

[14:20:06] Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


XGBoostError: b'[14:20:06] /workspace/src/objective/regression_obj.cu:65: Check failed: info.labels_.Size() != 0U (0 vs. 0) label set cannot be empty\n\nStack trace returned 10 entries:\n[bt] (0) /opt/conda/xgboost/libxgboost.so(dmlc::StackTrace()+0x3d) [0x7fb9897f85cd]\n[bt] (1) /opt/conda/xgboost/libxgboost.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x18) [0x7fb9897f89c8]\n[bt] (2) /opt/conda/xgboost/libxgboost.so(xgboost::obj::RegLossObj<xgboost::obj::LogisticClassification>::GetGradient(xgboost::HostDeviceVector<float> const&, xgboost::MetaInfo const&, int, xgboost::HostDeviceVector<xgboost::detail::GradientPairInternal<float> >*)+0xcd) [0x7fb9899ee28d]\n[bt] (3) /opt/conda/xgboost/libxgboost.so(xgboost::LearnerImpl::UpdateOneIter(int, xgboost::DMatrix*)+0x362) [0x7fb98986f1e2]\n[bt] (4) /opt/conda/xgboost/libxgboost.so(XGBoosterUpdateOneIter+0x35) [0x7fb9897f0ab5]\n[bt] (5) /opt/conda/lib/python3.6/lib-dynload/../../libffi.so.6(ffi_call_unix64+0x4c) [0x7fb9e0c32ec0]\n[bt] (6) /o

In [12]:
del total
gc.collect()

7

In [ ]:
pred_test = bst.predict(test)
test_acc = accuracy_score(test.get_label(), pred_test)
test_auc = roc_auc_score(test.get_label(), pred_test)

# LightGBM模型

## 准备训练数据

In [4]:
%%time

print(datetime.now())
# 一般特征
feature_normal = pd.read_csv('./handled_data/train_feature/train_feature_3kw_normal.csv.gz', header=None)
print('loaded normal feature     --->',feature_normal.shape)

# feature_statistics = pd.read_csv('./handled_data/train_feature/train_feature_3kw_statistics.csv.gz', header=None)
# print('loaded statistics feature --->',feature_statistics.shape)
# feature_pagerank = pd.read_csv('./handled_data/train_feature/train_feature_3kw_pagerank.csv.gz', header=None)
# print('loaded pagerank feature   --->',feature_pagerank.shape)

feature_graph = pd.read_csv('./handled_data/train_feature/train_feature_graph_final.csv.gz', header=None)
print('loaded graph feature      --->',feature_graph.shape)

feature_fuzzy = pd.read_csv('./handled_data/train_feature/train_feature_fuzzy.csv', header=None)
print('loaded fuzzy feature      --->',feature_fuzzy.shape)
feature_powerful = pd.read_csv('./handled_data/train_feature/train_feature_powerful.csv', header=None)
print('loaded powerful feature   --->',feature_powerful.shape)

# 融合特征
feat_dssm_300 = pd.read_csv('./predictions/feat_dssm_300_3kw.csv.gz', header=None)
print('loaded dssm_300 feature   --->', feat_dssm_300.shape)

feat_dssm_600 = pd.read_csv('./predictions/feat_dssm_600_3kw.csv.gz', header=None)
print('loaded dssm_600 feature   --->', feat_dssm_600.shape)   

feat_mvlstm = pd.read_csv('./predictions/feat_mvlstm_3kw.csv.gz', header=None)
print('loaded mvlstm feature     --->', feat_mvlstm.shape) 

feat_arc = pd.read_csv('./predictions/feat_arc_3kw.csv.gz', header=None)
print('loaded ARC feature        --->', feat_arc.shape) 
    
feat_textcnn = pd.read_csv('./predictions/feat_textcnn_3kw.csv.gz', header=None)
print('loaded TextCNN feature    --->', feat_textcnn.shape)

feat_oofcnn = pd.read_csv('./predictions/feat_oofcnn_3kw.csv', header=None)
print('loaded OOF CNN feature    --->', feat_oofcnn.shape)

feat_ooflstm = pd.read_csv('./predictions/feat_ooflstm_3kw.csv', header=None)
print('loaded OOF LSTM feature   --->', feat_ooflstm.shape)

label = pd.read_csv('./handled_data/label', header=None)
print('loaded label with shape   --->',label.shape)
print('----------------------end reading file--------------------------')
total = pd.concat([feature_normal, 
                    # feature_statistics, feature_pagerank, 
                    feature_graph,
                    feature_fuzzy, feature_powerful,
                    feat_dssm_300,
                    feat_dssm_600,
                    feat_mvlstm,
                    feat_arc,
                    feat_textcnn,
                    feat_oofcnn,
                    feat_ooflstm,
                    label], axis=1)
total.columns = range(total.shape[1])
print('total feature shape       --->', total.shape)

print('split train set and test set')
label_idx = total.shape[1] - 1

train, val = train_test_split(total, test_size=0.1, random_state=20, stratify=total[label_idx])

x = train.drop(label_idx, axis=1)
y = train[label_idx]

val_x = val.drop(label_idx, axis=1)
val_y = val[label_idx]

print('initialize dataset')
lgb_train = lgb.Dataset(x, y, free_raw_data=False)
lgb_eval = lgb.Dataset(val_x, val_y, reference=lgb_train, free_raw_data=False)

2019-08-11 19:19:30.037143
loaded normal feature     ---> (30000000, 31)
loaded graph feature      ---> (30000000, 5)
loaded fuzzy feature      ---> (30000000, 7)
loaded powerful feature   ---> (30000000, 2)
loaded dssm_300 feature   ---> (30000000, 1)
loaded dssm_600 feature   ---> (30000000, 1)
loaded mvlstm feature     ---> (30000000, 1)
loaded ARC feature        ---> (30000000, 1)
loaded TextCNN feature    ---> (30000000, 1)
loaded OOF CNN feature    ---> (30000000, 1)
loaded OOF LSTM feature   ---> (30000000, 1)
loaded label with shape   ---> (30000000, 1)
----------------------end reading file--------------------------
total feature shape       ---> (30000000, 53)
split train set and test set
initialize dataset
CPU times: user 8min 10s, sys: 40.9 s, total: 8min 51s
Wall time: 9min 9s


In [9]:
total_x = total.drop(label_idx, axis=1)
total_y = total[label_idx]
print(total_x.shape)
print(total_x.info())
print(total_y.shape)
pritn(total_y.info())
lgb_train = lgb.Dataset(total_x, total_y, free_raw_data=False)

(30000000, 52)
                 0             1             2             3              4   \
count  3.000000e+07  3.000000e+07  3.000000e+07  3.000000e+07   3.000000e+07   
mean   3.138709e-01  6.360163e-01  7.210984e-01  1.328846e+01   5.023723e-05   
std    1.871304e-01  1.825601e-01  1.542859e-01  4.734423e+00   7.655814e-04   
min    0.000000e+00  3.225806e-02  0.000000e+00  1.000000e+00  3.876599e-302   
25%    1.904762e-01  5.000000e-01  6.363636e-01  1.000000e+01   1.425644e-12   
50%    2.857143e-01  6.666667e-01  7.333333e-01  1.300000e+01   3.036743e-10   
75%    4.090909e-01  7.777778e-01  8.181818e-01  1.600000e+01   6.644035e-08   
max    3.902353e+01  1.000000e+00  1.000000e+00  3.850000e+02   6.000000e-01   

                 5             6             7             8             9   \
count  3.000000e+07  3.000000e+07  3.000000e+07  3.000000e+07  3.000000e+07   
mean   2.809463e-01  5.420982e-01  8.260438e-01  7.305404e-01 -7.067164e-01   
std    1.559342e-01  2.9225

## 训练

In [10]:
%%time
params = {
    'boosting': 'dart',
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': '0.5',
    
    'num_leaves':59,
    'max_depth':7,
    
    'max_bin':251,
    'min_data_in_leaf':19,
         
    'scale_pos_weight': 3.1,
         
    'lambda_l1':0,
    'lambda_l2':0,
    'min_split_gain':0,
    
    'device': 'gpu',
    'gpu_platform_id':0,
    'gpu_device_id':0
}

print(datetime.now())
gbm = lgb.train(params, lgb_train, num_boost_round=450, early_stopping_rounds=30)
gbm.save_model('./handled_data/final_lgb_450.model') 
# init_model=gbm, keep_training_booster=True

2019-08-11 19:36:41.955449
CPU times: user 9h 24min 49s, sys: 1min 7s, total: 9h 25min 56s
Wall time: 41min 38s


## GridSearch

In [14]:
# 6 64
# 7 58
# params = {'max_depth':range(4,8), 'num_leaves':range(56, 60)}
# 2 20
# params = {'min_child_samples':range(18,23)}

# params ={'max_bin': range(5,100,10), }

params = {'min_data_in_leaf':range(240, 261)}
gbm = lgb.LGBMClassifier(boost='dart',
                            max_depth=7, 
                            num_leaves=59,
                            max_bin = 95,
                            learning_rate=0.5, 
                            n_estimators=100,
                            metric='auc')
gridSearch = GridSearchCV(estimator=gbm, param_grid=params, scoring='roc_auc', 
                                    cv=5, verbose=1, n_jobs=15)

gridSearch.fit(x, y)

print(gridSearch.best_params_ , '\n', gridSearch.best_score_)

Fitting 5 folds for each of 21 candidates, totalling 105 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed: 34.0min
[Parallel(n_jobs=10)]: Done 105 out of 105 | elapsed: 120.1min finished


{'min_data_in_leaf': 251} 
 0.7039860009576545


## 查看特征重要性

In [11]:
name = ['concurrence', 'levenshteinDistance', 'sorensenDistance', 
        'sameWord', 'specialConcurrence',
        'diceDistance', 'ochiaiDistance', 'jaccardDistance', 
        'word2vecDistance_1', 'word2vecDistance_2', 'word2vecDistance_3', 
        'fasttextDistance_1', 'fasttextDistance_2', 'fasttextDistance_3', 
        'w2vWeightDistance_1', 'w2vWeightDistance_2', 'w2vWeightDistance_3',
        'w2vKeywordDistance_1', 'w2vKeywordDistance_2', 'w2vKeywordDistance_3',
        'TFIDFDistance_1', 'TFIDFDistance_2', 'TFIDFDistance_3', 
        'TFIDFDistance_4', 'TFIDFDistance_5', 
        'fasttextWeightDistance_1', 'fasttextWeightDistance_2', 'fasttextWeightDistance_3',
        'fasttextKeywordDistance_1', 'fasttextKeywordDistance_2', 'fasttextKeywordDistance_3',
        'seq1_max_degree', 'seq2_max_degree', 'seq1_max_component',
        'pagerank_feature1', 'pagerank_feature2',
        'fuzzy_1', 'fuzzy_2', 'fuzzy_3', 'fuzzy_4', 'fuzzy_5', 'word2vec', 'fasttext',
        'singleSide', 'doubleSide',
        # 'seq1_neighbor', 'seq2_neighbor', 'total_neighbor',
        'DSSM_300', 'DSSM_600', 
        'MVLSTM', 'ARC','TextCNN','OOFCNN' ,'OOFLSTM' ]
        
print(len(name))
print(gbm.feature_importance().__len__())
feature_importance = pd.DataFrame({'feature_name': name[:gbm.feature_importance().__len__()], 
                                'importance': gbm.feature_importance()}).sort_values(by='importance', ascending=False).reset_index(drop = True)
print(feature_importance)
for i in range(len(gbm.feature_importance())):
    print(i, name[i], gbm.feature_importance()[i])

52
52
                 feature_name  importance
0                    DSSM_300        1522
1             seq1_max_degree        1460
2           pagerank_feature2        1374
3                     fuzzy_1        1246
4           pagerank_feature1        1182
5                     fuzzy_3        1154
6                     fuzzy_4        1119
7                  doubleSide         890
8                    sameWord         750
9                      OOFCNN         743
10         fasttextDistance_1         611
11        levenshteinDistance         590
12                    fuzzy_5         562
13            TFIDFDistance_2         550
14         specialConcurrence         519
15                   DSSM_600         512
16                    fuzzy_2         476
17         word2vecDistance_1         448
18            seq2_max_degree         430
19   fasttextWeightDistance_2         425
20                        ARC         399
21   fasttextWeightDistance_1         397
22                 singleSid

## 测试结果

In [12]:
%%time
print(datetime.now())
# gbm = lgb.Booster(model_file='./handled_data/lgb_2kw_650.model')
pred = gbm.predict(val_x, num_iteration=gbm.best_iteration)

print('end predict')
result = []
for item in pred:
    if item>=0.5:
        result.append(1)
    else:
        result.append(0)
count = 0
pos_count = 0
print('pos_predict:',np.sum(result))

val_y_list = val_y.tolist()
for i in range(len(pred)):
    if val_y_list[i] == 1 and result[i] == val_y_list[i]:
        pos_count += 1
    if result[i] == val_y_list[i]:
        count+=1
print('acc:',count/len(pred))
print('pos_right_count:',pos_count)
print('pos_label:',np.sum(val_y.tolist()))
print('pos_acc:',pos_count / np.sum(val_y.tolist()))

2019-08-11 20:18:38.544223


NameError: name 'val_x' is not defined

## 提交文件

In [13]:
%%time

gbm = lgb.Booster(model_file='./handled_data/final_lgb_450.model')
feature_normal_test = pd.read_csv('./handled_data/test_feature/test_feature_3kw_normal.csv', header=None)
print('loaded feature normal     --->',feature_normal_test.shape)
feature_statistics_test = pd.read_csv('./handled_data/test_feature/test_feature_3kw_statistics.csv', header=None)
print('loaded feature statistics --->',feature_statistics_test.shape)
feature_pagerank_test = pd.read_csv('./handled_data/test_feature/test_feature_3kw_pagerank.csv', header=None)
print('loaded feature pagerank   --->',feature_pagerank_test.shape)
feature_fuzzy_test = pd.read_csv('./handled_data/test_feature/test_feature_fuzzy.csv', header=None)
print('loaded fuzzy feature      --->',feature_fuzzy_test.shape)
feature_powerful_test = pd.read_csv('./handled_data/test_feature/test_feature_powerful.csv', header=None)
print('loaded powerfule feature  --->',feature_powerful_test.shape)

# 融合特征
feat_dssm_300_test = pd.read_csv('./predictions/test_dssm_300_2kw.csv.gz', header=None)
print('loaded feature dssm_300   --->', feat_dssm_300_test.shape)
feat_dssm_600_test = pd.read_csv('./predictions/test_dssm_600_2kw.csv.gz', header=None)
print('loaded feature dssm_600   --->', feat_dssm_600_test.shape)
feat_mvlstm_test = pd.read_csv('./predictions/test_mv_2kw.csv.gz', header=None)
print('loaded feature MVLSTM     --->', feat_mvlstm_test.shape)
feat_arc_test = pd.read_csv('./predictions/test_arc_2kw.csv.gz', header=None)
print('loaded feature ARC        --->', feat_arc_test.shape)
feat_textcnn_test = pd.read_csv('./predictions/test_textcnn_2kw.csv.gz', header=None)
print('loaded feature TextCNN    --->', feat_textcnn_test.shape)

# feat_lstm_dssm_test = pd.read_csv('./predictions/test_lstm_dssm_2kw.csv', header=None)
# print('loaded feature LSTM-DSSM  --->', feat_lstm_dssm_test.shape)

print('------------finish reading file, concating feature------------')
test_feature = pd.concat([feature_normal_test, feature_statistics_test, feature_pagerank_test, 
                feature_fuzzy_test, feature_powerful_test, 
                feat_dssm_300_test,
                feat_dssm_600_test,
                feat_mvlstm_test,
                feat_arc_test,
                feat_textcnn_test,
                # feat_lstm_dssm_test
                ], axis=1)
print('concated feature, shape    --->', test_feature.shape)

LightGBMError: Could not open ./handled_data/lgb_power_450.model

In [7]:
%%time
pred_test = pd.DataFrame(gbm.predict(test_feature, num_iteration=gbm.best_iteration))

test_data = pd.read_csv(test_data_path, header=None)
submission = pd.concat([test_data[0], test_data[2], pred_test], axis=1)
print('finish prediction, shpae   --->', submission.shape)
submission.to_csv('./predictions/lgb_submission_power_450.csv', header=None, index=False)

finish prediction, shpae   ---> (20000000, 3)


In [10]:
!https_proxy="http://klab-external-proxy" ./kesci_submit -file ./predictions/lgb_submission_power_450.csv -token 8be4f72dc2395a8d

Kesci Submit Tool 3.2.1

> 已验证Token
> 提交文件 ./predictions/lgb_submission_power_450.csv (565534.77 KiB)
> 已上传 100 %
> 文件已上传        
> 服务器响应: 200 提交成功，请等待评审完成
> 提交完成


## Final提交文件

In [4]:
# !wc -l /home/kesci/work/predictions/testfin_mvlstm.csv
# !wc -l /home/kesci/work/predictions/testfin_textcnn.csv
!killall python

In [6]:
%%time
# final = pd.read_csv(final_data_path, header=None)
# result = pd.read_csv('./predictions/final_submission_20190811.csv', header=None)
print(final[0].unique())
print(result[0].unique())
print(final[0].unique().__len__())
print(result[0].unique().__len__())
print()

print(final[2].unique())
print(result[1].unique())
print(final[2].unique().__len__())
print(result[1].unique().__len__())

[       1        2        3 ... 10318080 10318081 10318082]
[       1        2        3 ... 10318080 10318081 10318082]
10318082
10318082

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
20
20
CPU times: user 8.47 s, sys: 2.29 s, total: 10.8 s
Wall time: 11 s


In [3]:
%%time
print(datetime.now())
CHUNKSIZE = 10000000
gbm = lgb.Booster(model_file='./handled_data/final_lgb_450.model')

feature_normal_final = pd.read_csv('./handled_data/final/final_feature_normal.csv', header=None, chunksize=CHUNKSIZE)
feature_statistics_final = pd.read_csv('./handled_data/final/final_feature_statistics.csv', header=None, chunksize=CHUNKSIZE)
feature_pagerank_final = pd.read_csv('./handled_data/final/final_feature_pagerank.csv', header=None, chunksize=CHUNKSIZE)
feature_fuzzy_final = pd.read_csv('./handled_data/final/final_feature_fuzzy.csv', header=None, chunksize=CHUNKSIZE)
feature_powerful_final = pd.read_csv('./handled_data/final/final_feature_powerful.csv', header=None, chunksize=CHUNKSIZE)

feat_dssm_300_final = pd.read_csv('./predictions/testfin_300_dssm.csv_10kw.gz', header=None, chunksize=CHUNKSIZE)
feat_dssm_600_final = pd.read_csv('./predictions/testfin_600_dssm.csv', header=None, chunksize=CHUNKSIZE)
feat_mvlstm_final = pd.read_csv('./predictions/testfin_mvlstm.csv', header=None, chunksize=CHUNKSIZE)
feat_arc_final = pd.read_csv('./predictions/testfin_arc.csv', header=None, chunksize=CHUNKSIZE)
feat_textcnn_final = pd.read_csv('./predictions/testfin_textcnn.csv', header=None, chunksize=CHUNKSIZE)
feat_oofcnn_final = pd.read_csv('./predictions/testfin_oofcnn.csv', header=None, chunksize=CHUNKSIZE)
feat_ooflstm_final = pd.read_csv('./predictions/testfin_ooflstm.csv', header=None, chunksize=CHUNKSIZE)
final_data = pd.read_csv(final_data_path, header=None, chunksize=CHUNKSIZE)
result = []
print('start iteration')
for i in tqdm(range(10)):
    features = pd.concat([feature_normal_final.get_chunk(), 
                            feature_statistics_final.get_chunk(), feature_pagerank_final.get_chunk(),
                            feature_fuzzy_final.get_chunk(), feature_powerful_final.get_chunk(),
                            feat_dssm_300_final.get_chunk(),
                            feat_dssm_600_final.get_chunk(),
                            feat_mvlstm_final.get_chunk(),
                            feat_arc_final.get_chunk(),
                            feat_textcnn_final.get_chunk(),
                            feat_oofcnn_final.get_chunk(),
                            feat_ooflstm_final.get_chunk()], axis=1)
                            
    idx = final_data.get_chunk()
    pred = gbm.predict(features, num_iteration=gbm.best_iteration)
    result.append(pd.DataFrame({'query_id': idx[0].tolist(), 'title_id': idx[2].tolist(), 'label': pred}))
    print(result[i].shape)
    
    del pred
    del idx
    del features
    gc.collect()
    
result = pd.concat(result)
result.columns = range(result.shape[1])
print(result.describe())
print('finish prediction, shpae   --->', result.shape)
result.to_csv('./predictions/final_submission_20190811_2.csv', header=None, index=False)

# feature_append_test = pd.read_csv('./handled_data/final/test_feature_append.csv', header=None)
# print('loaded append feature     --->',feature_append_test.shape)

# 融合特征
# with gzip.open('./predictions/testfin_300_dssm.csv_10kw.gz', 'rb') as f:
#     feat_dssm_300_final = pd.read_csv(f, header=None)
# print('loaded feature dssm_300   --->', feat_dssm_300_final.shape)
    
# with gzip.open('./predictions/testfin_600_dssm.csv_10kw.gz', 'rb') as f:
#     feat_dssm_600_final = pd.read_csv(f, header=None)
# feat_dssm_600_final = pd.read_csv('./predictions/testfin_600_dssm_p1.csv', header=None)
# print('loaded feature dssm_600   --->', feat_dssm_600_final.shape)
    
# with gzip.open('./predictions/test_mv_2kw.csv.gz', 'rb') as f:
#     feat_mvlstm_test = pd.read_csv(f, header=None)
# print('loaded feature MVLSTM     --->', feat_mvlstm_test.shape)
    
# with gzip.open('./predictions/test_arc_2kw.csv.gz', 'rb') as f:
#     feat_arc_test = pd.read_csv(f, header=None)
# print('loaded feature ARC        --->', feat_arc_test.shape)
    
# with gzip.open('./predictions/test_textcnn_2kw.csv.gz', 'rb') as f:
#     feat_textcnn_test = pd.read_csv(f, header=None)
# print('loaded feature TextCNN    --->', feat_textcnn_test.shape)

# feat_lstm_dssm_test = pd.read_csv('./predictions/test_lstm_dssm_2kw.csv', header=None)
# print('loaded feature LSTM-DSSM  --->', feat_lstm_dssm_test.shape)

  0%|          | 0/10 [00:00<?, ?it/s]

2019-08-11 20:20:04.433063
start iteration


 10%|█         | 1/10 [04:20<39:01, 260.19s/it]

(10000000, 3)


 20%|██        | 2/10 [08:31<34:19, 257.43s/it]

...

 90%|█████████ | 9/10 [37:57<04:12, 252.71s/it]

(10000000, 3)


100%|██████████| 10/10 [42:09<00:00, 252.36s/it]

(10000000, 3)


                  0             1             2
count  1.000000e+08  1.000000e+08  1.000000e+08
mean   5.170643e+06  7.224337e+00  4.130191e-01
std    2.967558e+06  5.240233e+00  1.687013e-01
min    1.000000e+00  1.000000e+00  0.000000e+00
25%    2.606632e+06  3.000000e+00  2.880982e-01
50%    5.195137e+06  6.000000e+00  4.040706e-01
75%    7.727133e+06  1.100000e+01  5.317806e-01
max    1.031808e+07  2.000000e+01  1.000000e+00
finish prediction, shpae   ---> (100000000, 3)
CPU times: user 2h 26min 37s, sys: 4min 11s, total: 2h 30min 49s
Wall time: 50min 28s


In [4]:
!https_proxy="http://klab-external-proxy" ./kesci_submit -file ./predictions/final_submission_20190811_2.csv -token 8be4f72dc2395a8d -mode archive            

Kesci Submit Tool 3.2.1

> 已验证Token
> 提交文件 ./predictions/final_submission_20190811_2.csv (2877771.46 KiB)
> 已上传 100 %
> 文件已上传        
> 服务器响应: 200 提交成功
> 提交完成


# end